<a href="https://colab.research.google.com/github/sandvenu/GENAI-/blob/main/HYBRID_RAG_APP_Using_LLamaindex_and_Mistral.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# HYBRID SERACH RAG application



In [ ]:
%pip install llama-index-llms-huggingface
%pip install llama-index

In [ ]:
!pip install weaviate-client

In [3]:
WEAVIATE_CLUSTER="https://my-box-ynfohobo.weaviate.network"
WEAVIATE_API_KEY="pimfsk5Bmul1U4Bn1rYNFI5MypJZ9OitvoOA"


In [4]:

import weaviate
client=weaviate.Client(url=WEAVIATE_CLUSTER,auth_client_secret=weaviate.AuthApiKey(WEAVIATE_API_KEY))

In [5]:
client=weaviate.Client(url=WEAVIATE_CLUSTER,auth_client_secret=weaviate.AuthApiKey(WEAVIATE_API_KEY))

In [6]:
from llama_index.core import VectorStoreIndex, SimpleDirectoryReader


In [7]:
!mkdir Data

In [8]:
loader=SimpleDirectoryReader('/content/Data')

In [9]:
documents=loader.load_data()

In [ ]:
documents

In [ ]:
%pip install llama-index-embeddings-huggingface
%pip install llama-index-embeddings-instructor

In [ ]:
from llama_index.embeddings.huggingface import HuggingFaceEmbedding
embed_model=HuggingFaceEmbedding(model_name="sentence-transformers/all-mpnet-base-v2")


In [12]:
from llama_index.core import PromptTemplate

In [13]:
from llama_index.llms.huggingface import HuggingFaceLLM

In [14]:
system_prompt=""""<|SYSTEM|># You are a Q&A assistant. Your goal is to answer questions as
accurately as possible based on the instructions and context provided

"""

In [17]:
# This will wrap the default prompts that are internal to llama-index
query_wrapper_prompt = PromptTemplate("<|USER|>{query_str}<|ASSISTANT|>")

In [ ]:
!huggingface-cli login

model_name: str = Field(
        default=DEFAULT_HUGGINGFACE_MODEL,
        description=(
            "The model name to use from HuggingFace. "
            "Unused if `model` is passed in directly."
        ),
    )
    context_window: int = Field(
        default=DEFAULT_CONTEXT_WINDOW,
        description="The maximum number of tokens available for input.",
        gt=0,
    )
    max_new_tokens: int = Field(
        default=DEFAULT_NUM_OUTPUTS,
        description="The maximum number of tokens to generate.",
        gt=0,
    )
    system_prompt: str = Field(
        default="",
        description=(
            "The system prompt, containing any extra instructions or context. "
            "The model card on HuggingFace should specify if this is needed."
        ),
    )
    query_wrapper_prompt: PromptTemplate = Field(
        default=PromptTemplate("{query_str}"),
        description=(
            "The query wrapper prompt, containing the query placeholder. "
            "The model card on HuggingFace should specify if this is needed. "
            "Should contain a `{query_str}` placeholder."
        ),
    )
    tokenizer_name: str = Field(
        default=DEFAULT_HUGGINGFACE_MODEL,
        description=(
            "The name of the tokenizer to use from HuggingFace. "
            "Unused if `tokenizer` is passed in directly."
        ),
    )
    device_map: str = Field(
        default="auto", description="The device_map to use. Defaults to 'auto'."
    )
    stopping_ids: List[int] = Field(
        default_factory=list,
        description=(
            "The stopping ids to use. "
            "Generation stops when these token IDs are predicted."
        ),
    )
    tokenizer_outputs_to_remove: list = Field(
        default_factory=list,
        description=(
            "The outputs to remove from the tokenizer. "
            "Sometimes huggingface tokenizers return extra inputs that cause errors."
        ),
    )
    tokenizer_kwargs: dict = Field(
        default_factory=dict, description="The kwargs to pass to the tokenizer."
    )
    model_kwargs: dict = Field(
        default_factory=dict,
        description="The kwargs to pass to the model during initialization.",
    )
    generate_kwargs: dict = Field(
        default_factory=dict,
        description="The kwargs to pass to the model during generation.",
    )
    is_chat_model: bool = Field(
        default=False,
        description=(
            LLMMetadata.__fields__["is_chat_model"].field_info.description
            + " Be sure to verify that you either pass an appropriate tokenizer "
            "that can convert prompts to properly formatted chat messages or a "
            "`messages_to_prompt` that does so."
        ),
    )

In [18]:
import torch
llm=HuggingFaceLLM(
    context_window=4096,
    max_new_tokens=256,
    generate_kwargs={"temperature": 0.7, "do_sample": False},
    system_prompt=system_prompt,
    query_wrapper_prompt=query_wrapper_prompt,
    tokenizer_name="mistralai/Mistral-7B-Instruct-v0.1",
    model_name="mistralai/Mistral-7B-Instruct-v0.1",
    device_map="auto",
    stopping_ids=[50278, 50279, 50277, 1, 0],
    tokenizer_kwargs={"max_length": 4096},
    # uncomment this if using CUDA to reduce memory usage
    model_kwargs={"torch_dtype": torch.float16}

)

config.json:   0%|          | 0.00/571 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/25.1k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/9.94G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/4.54G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/1.47k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/493k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.80M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/72.0 [00:00<?, ?B/s]

In [19]:
from llama_index.core import ServiceContext

Service_Context=ServiceContext.from_defaults(
    chunk_size=1024,
    llm=llm,
    embed_model=embed_model
)

<ipython-input-19-f0b118d025f9>:3: DeprecationWarning: Call to deprecated class method from_defaults. (ServiceContext is deprecated, please use `llama_index.settings.Settings` instead.) -- Deprecated since version 0.10.0.
  Service_Context=ServiceContext.from_defaults(


In [20]:
from llama_index.core import VectorStoreIndex

In [21]:
WEAVIATE_CLUSTER="https://my-box-ynfohobo.weaviate.network"
WEAVIATE_API_KEY="pimfsk5Bmul1U4Bn1rYNFI5MypJZ9OitvoOA"
import weaviate
client=weaviate.Client(url=WEAVIATE_CLUSTER,auth_client_secret=weaviate.AuthApiKey(WEAVIATE_API_KEY))

In [22]:
!pip install llama-index-vector-stores-weaviate

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 120.4/120.4 kB 1.6 MB/s eta 0:00:00
  Attempting uninstall: weaviate-client
    Found existing installation: weaviate-client 4.5.6
    Uninstalling weaviate-client-4.5.6:
      Successfully uninstalled weaviate-client-4.5.6


In [23]:
from llama_index.vector_stores.weaviate import WeaviateVectorStore

In [24]:
from llama_index.core import StorageContext

In [25]:
vector_store=WeaviateVectorStore(weaviate_client=client,index_name="Llamaindex")

In [26]:
storage_context = StorageContext.from_defaults(vector_store=vector_store)


In [ ]:
index = VectorStoreIndex.from_documents(
    documents, storage_context=storage_context,embed_model=embed_model
)

In [ ]:
query_engine = index.as_query_engine(llm=llm)

In [ ]:
result3=query_engine.query("what is attention?")

/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:492: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.7` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


In [ ]:
result3.response

'Attention is a mechanism used in machine translation models to retain and utilize all the hidden state of the input sequence during the decoding phase. It allows the model to focus and pay more attention to the relevant part of the input sequence when producing the output. The attention mechanism just addresses the issue of the encoder not being able to memorize the words coming at the beginning of the sentences, which leads to poor translation of the source sentence. During the decoding phase, the model creates an alignment between each time step of the decoder output and all of the encoder hidden state. We need to learn this alignment. Each output of the decoder can selectively pick out specific elements from the sequence to produce the output. This allows the model to focus and pay more attention to the relevant part of the input sequence.'

In [27]:
from llama_index.core.node_parser import SimpleNodeParser

node_parser = SimpleNodeParser.from_defaults(chunk_size=1024)

# Extract nodes from documents
nodes = node_parser.get_nodes_from_documents(documents)

In [28]:
import weaviate

# Connect to your Weaviate instance
client = weaviate.Client(
    embedded_options=weaviate.embedded.EmbeddedOptions(),
)

Binary /root/.cache/weaviate-embedded did not exist. Downloading binary from https://github.com/weaviate/weaviate/releases/download/v1.23.7/weaviate-v1.23.7-Linux-amd64.tar.gz
Started /root/.cache/weaviate-embedded: process ID 2962


In [32]:
from llama_index.core import VectorStoreIndex, StorageContext
from llama_index.vector_stores.weaviate import WeaviateVectorStore
from llama_index.core import Settings

index_name = "MyExternalContext"

# Construct vector store
vector_store = WeaviateVectorStore(
    weaviate_client = client,
    index_name = index_name
)

# Set up the storage for the embeddings
storage_context = StorageContext.from_defaults(vector_store=vector_store)

# Setup the index
# build VectorStoreIndex that takes care of chunking documents
# and encoding chunks to embeddings for future retrieval
index = VectorStoreIndex(nodes,storage_context=storage_context,embed_model=embed_model )

In [34]:
# The QueryEngine class is equipped with the generator
# and facilitates the retrieval and generation steps
query_engine = index.as_query_engine(llm=llm)

In [35]:
# Run your naive RAG query
result4 = query_engine.query(
    "What is attention?"
)

/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:492: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.7` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


In [36]:
result4.response

'Attention is a mechanism used in machine translation models to retain and utilize all the hidden state of the input sequence during the decoding phase. It allows the model to focus and pay more attention to the relevant part of the input sequence when producing the output. The attention mechanism just addresses the issue of the encoder not being able to memorize the words coming at the beginning of the sentences, which leads to poor translation of the source sentence. During the decoding phase, the model creates an alignment between each time step of the decoder output and all of the encoder hidden state. We need to learn this alignment. Each output of the decoder can selectively pick out specific elements from the sequence to produce the output. This allows the model to focus and pay more attention to the relevant part of the input sequence.'

In [37]:
#Run your Hybrid search
query_engine = index.as_query_engine(
    llm=llm,
    vector_store_query_mode="hybrid",
    alpha=0.5,
)

In [38]:
# Run your naive RAG query
result5 = query_engine.query(
    "What is attention?"
)

/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:492: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.7` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


In [39]:
result5.response

'Attention is a mechanism used in neural machine translation to retain and utilize all the hidden state of the input sentence during the decoding phase. It allows the model to focus and pay more attention to the relevant part of the input sequence. During the decoding phase, the model creates an alignment between each time step of the decoder output and all of the encoder hidden state. This alignment is learned by the model, and each output of the decoder can selectively pick out specific elements from the sequence to produce the output. This allows the model to focus and pay more attention to the relevant part of the input sequence.'

In [ ]:
#Post-retrieval optimization Re-ranking
#!pip install torch sentence-transformers
from llama_index.core.postprocessor import SentenceTransformerRerank

# Define reranker model
rerank = SentenceTransformerRerank(
    top_n = 2,
    model = "BAAI/bge-reranker-base"
)

# Add reranker to query engine
query_engine = index.as_query_engine(
		similarity_top_k = 6,
		llm=llm,
    node_postprocessors = [rerank],
)

In [41]:
# Run your naive RAG query
result6 = query_engine.query(
    "What is attention?"
)

/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:492: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.7` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


In [42]:
result6.response

'Attention is a mechanism used in machine translation models to retain and utilize all the hidden state of the input sentence during the decoding phase. It allows the model to focus and pay more attention to the relevant part of the input sequence, which helps to improve the translation of the source sentence. Attention is a way to learn the alignment between the source word and the target word, which can vary from language to language. It is a way to address the problem of the encoder not being able to memorize the words coming at the beginning of the sentences, which leads to poor translation of the source sentence.'